In [2]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=2)
today, yesterday

(datetime.date(2022, 4, 2), datetime.date(2022, 3, 31))

In [3]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 4, 1), datetime.date(2022, 3, 25))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [4]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

3278

In [5]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AIMIRT', 'ASIAN', 'BAM', 'BBL', 'BCH', 'CPNCG', 'DOHOME', 'HREIT', 'JASIF', 'KKP', 'MAKRO', 'ORI', 'PTT', 'RATCH', 'SC', 'SCC', 'SENA', 'SPALI', 'STA', 'TMT'"

### Get past one year data

In [6]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-03-31' AND name IN ('AIMIRT', 'ASIAN', 'BAM', 'BBL', 'BCH', 'CPNCG', 'DOHOME', 'HREIT', 'JASIF', 'KKP', 'MAKRO', 'ORI', 'PTT', 'RATCH', 'SC', 'SCC', 'SENA', 'SPALI', 'STA', 'TMT') 
ORDER BY name, date


In [7]:
df = pd.read_sql(sql, const)
df.shape

(4929, 7)

In [8]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [9]:
sql = """
SELECT name,trade
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,trade
name,
AIMIRT,S
ASIAN,B
BAM,B
BBL,B
BCH,B
CPNCG,S
DOHOME,B
HREIT,S
JASIF,S


### Create monitors from stocks

In [10]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,trade
name,
AIMIRT,S
ASIAN,B
BAM,B
BBL,B
BCH,B
CPNCG,S
DOHOME,B
HREIT,S
JASIF,S


In [11]:
monitors.shape[0]

20

In [12]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [13]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders 
WHERE name IN (%s)
GROUP BY trade
ORDER BY trade
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,13
1,S,7


In [14]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [15]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(126, 1)

In [16]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(127, 3)

In [17]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge
14,SC,B,left_only


In [18]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(107, 3)

In [19]:
sql = """
SELECT name, trade, market
FROM orders 
ORDER BY name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,trade,market
name,,
AIMIRT,S,SET
ASIAN,B,SET
BAM,B,SET100
BBL,B,SET50
BCH,B,SET100
CPNCG,S,SET
DOHOME,B,SET100
HREIT,S,SET
JASIF,S,SET


In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
ASIAN,B
BAM,B
BBL,B
BCH,B
DOHOME,B
KKP,B
MAKRO,B
ORI,B
SC,B


In [22]:
buy_candidates.shape[0]

13

In [23]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AIMIRT,S
CPNCG,S
HREIT,S
JASIF,S
PTT,S
RATCH,S
STA,S


In [24]:
sell_candidates.shape[0]

7